In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
seed = 42
np.random.seed(seed)
import tensorflow as tf
from tensorflow import keras
from transformers import TFDistilBertModel, DistilBertTokenizer


In [ ]:
MODEL_NAME = 'distilbert-base-uncased'  
MAX_LEN = 128  
BATCH_SIZE = 16  
GRADIENT_ACCUMULATION_STEPS = 2  
LEARNING_RATE = 2e-5  
WARMUP_STEPS = 100  
EPOCHS = 10  
VALIDATION_SPLIT = 0.2
CLASSIFIER_DROPOUT = 0.3  
DENSE_UNITS = 128  
L2_REG = 1e-4  
FREEZE_BASE = True  
UNFREEZE_LAST_N_LAYERS = 2  


In [ ]:
train = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/train.csv", index_col="row_id")
train.head(10)

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
sample_text = train['text'].iloc[0]
encoded = tokenizer.encode_plus(
    sample_text,
    add_special_tokens=True,  
    max_length=MAX_LEN,
    padding='max_length',  
    truncation=True,  
    return_attention_mask=True,  
    return_tensors='tf'  
)


In [ ]:
def tokenize_texts(texts, tokenizer, max_len):
    """
    Tokeniza una lista de textos con el tokenizer de DistilBERT
    Devuelve los input_ids y las attention_masks
    """
    input_ids = []
    attention_masks = []
    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids), np.array(attention_masks)
X_train_text = train['text'].values
y_train = train['spam_label'].values
X_train_ids, X_train_masks = tokenize_texts(X_train_text, tokenizer, MAX_LEN)
X_train_ids_final, X_val_ids, X_train_masks_final, X_val_masks, y_train_final, y_val = train_test_split(
    X_train_ids, X_train_masks, y_train,
    test_size=VALIDATION_SPLIT,
    random_state=seed,
    stratify=y_train
)


In [ ]:
class DistilBertSpamClassifier(keras.Model):
    def __init__(self, model_name, dense_units, dropout_rate, l2_reg, freeze_base=True):
        super(DistilBertSpamClassifier, self).__init__()
        self.distilbert = TFDistilBertModel.from_pretrained(model_name)
        if freeze_base:
            self.distilbert.trainable = False
        self.pooling = GlobalAveragePooling1D()  
        self.dense1 = Dense(
            dense_units,
            activation='relu',
            kernel_regularizer=l2(l2_reg),  
            name='dense_1'
        )
        self.dropout = Dropout(dropout_rate)
        self.output_layer = Dense(1, activation='sigmoid', name='output')
    def call(self, inputs, training=False):
        input_ids, attention_mask = inputs
        distilbert_output = self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            training=training
        )
        sequence_output = distilbert_output.last_hidden_state
        pooled = self.pooling(sequence_output)
        x = self.dense1(pooled)
        x = self.dropout(x, training=training)
        output = self.output_layer(x)
        return output
model = DistilBertSpamClassifier(
    model_name=MODEL_NAME,
    dense_units=DENSE_UNITS,
    dropout_rate=CLASSIFIER_DROPOUT,
    l2_reg=L2_REG,
    freeze_base=FREEZE_BASE
)
dummy_input_ids = tf.zeros((1, MAX_LEN), dtype=tf.int32)
dummy_attention_mask = tf.zeros((1, MAX_LEN), dtype=tf.int32)
_ = model([dummy_input_ids, dummy_attention_mask], training=False)
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc')
    ]
)
model.summary()
trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])
non_trainable_params = model.count_params() - trainable_params
distilbert_layer = model.distilbert

In [ ]:
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        'best_distilbert_spam_model.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-7,
        verbose=1
    )
]
history_phase1 = model.fit(
    [X_train_ids_final, X_train_masks_final],
    y_train_final,
    batch_size=BATCH_SIZE,
    epochs=3,
    validation_data=([X_val_ids, X_val_masks], y_val),
    callbacks=callbacks,
    verbose=1
)


In [ ]:
if FREEZE_BASE:
    distilbert_layer.trainable = True
    transformer_layers = distilbert_layer.distilbert.transformer.layer
    total_transformer_layers = len(transformer_layers)
    layers_to_freeze = max(0, total_transformer_layers - UNFREEZE_LAST_N_LAYERS)
    distilbert_layer.distilbert.embeddings.trainable = False
    for i, layer in enumerate(transformer_layers):
        if i < layers_to_freeze:
            layer.trainable = False
        else:
            layer.trainable = True
    optimizer_phase2 = keras.optimizers.Adam(learning_rate=LEARNING_RATE / 10)
    model.compile(
        optimizer=optimizer_phase2,
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall'),
            keras.metrics.AUC(name='auc')
        ]
    )
    trainable_params_phase2 = sum([tf.size(w).numpy() for w in model.trainable_weights])
    history_phase2 = model.fit(
        [X_train_ids_final, X_train_masks_final],
        y_train_final,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        initial_epoch=3,  
        validation_data=([X_val_ids, X_val_masks], y_val),
        callbacks=callbacks,
        verbose=1
    )
    history = history_phase1
    for key in history_phase2.history:
        history.history[key].extend(history_phase2.history[key])
else:
    history = history_phase1


In [ ]:
y_pred_proba = model.predict([X_val_ids, X_val_masks], batch_size=BATCH_SIZE)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()
mcc_score = matthews_corrcoef(y_val, y_pred)
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_val, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Not SPAM', 'SPAM'],
            yticklabels=['Not SPAM', 'SPAM'])
plt.title(f'DistilBERT Confusion Matrix (MCC: {mcc_score:.4f})')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
overfitting_delta = abs(final_val_loss - final_train_loss)
comparison_data = {
    'V1 LSTM': {'mcc': 0.8665, 'arch': 'Bi-LSTM(128)', 'params': '1.25M'},
    'V2 LSTM': {'mcc': 0.8885, 'arch': 'Bi-LSTM(96)+L2', 'params': '1.16M'},
    'V3 LSTM': {'mcc': 0.8733, 'arch': 'Bi-LSTM(64)+L2x5', 'params': '~1.0M'},
    'V4 DistilBERT': {'mcc': mcc_score, 'arch': 'DistilBERT+FT', 'params': f'{model.count_params()/1e6:.1f}M'}
}
for version, data in comparison_data.items():
best_lstm = 0.8885  
improvement = mcc_score - best_lstm
if mcc_score > 0.90:
elif mcc_score > best_lstm:
elif mcc_score > best_lstm - 0.01:
else:


In [ ]:
test = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/test.csv", index_col="row_id")
X_test_text = test['text'].values
X_test_ids, X_test_masks = tokenize_texts(X_test_text, tokenizer, MAX_LEN)
y_pred_proba_test = model.predict([X_test_ids, X_test_masks], batch_size=BATCH_SIZE)
y_pred_test = (y_pred_proba_test > 0.5).astype(int).flatten()


In [ ]:
submission = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/sample_submission.csv")
submission["spam_label"] = y_pred_test
submission.to_csv('submission.csv', index=False)
submission.head(10)

In [ ]:
summary_df = pd.DataFrame({
    'Iteración': ['V1', 'V2', 'V3', 'V4'],
    'Arquitectura': ['Bi-LSTM(128)', 'Bi-LSTM(96)+L2', 'Bi-LSTM(64)+L2x5', 'DistilBERT+FT'],
    'Val MCC': [0.8665, 0.8885, 0.8733, f'{mcc_score:.4f}'],
    'Parámetros': ['1.25M', '1.16M', '~1.0M', f'{model.count_params()/1e6:.1f}M'],
    'Trainable': ['1.25M', '1.16M', '~1.0M', f'{trainable_params/1e6:.1f}M'],
    'Approach': ['Baseline', 'Regularization', 'Shock Therapy', 'Transfer Learning']
})
